In [3]:
from transformers import BertForSequenceClassification

def get_bert_classifier(num_labels=2, model_name="bert-base-uncased"):
    """
    Load a BERT sequence classification model.

    Args:
        num_labels (int): Number of output classes.
        model_name (str): Pretrained model identifier.

    Returns:
        model (BertForSequenceClassification): BERT classifier ready for training.
    """
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    return model

# Example usage:
model = get_bert_classifier(num_labels=2)  # For binary classification
print(model.config)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [2]:
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix,
    precision_score, recall_score, classification_report
)
import torch

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)

    # Print results
    print(f"\n📊 Evaluation Results:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    print("\nClassification Report:\n", report)

    return acc, f1, precision, recall, cm




Baseline BERT 

In [1]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Load and balance Yelp
yelp_dataset = load_dataset("yelp_polarity")
class_0 = yelp_dataset["train"].filter(lambda ex: ex["label"] == 0).select(range(25000))
class_1 = yelp_dataset["train"].filter(lambda ex: ex["label"] == 1).select(range(25000))
balanced_train = concatenate_datasets([class_0, class_1])
balanced_yelp = DatasetDict({"train": balanced_train, "test": yelp_dataset["test"]})

# Tokenize
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_yelp(example): return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
tokenized_yelp = balanced_yelp.map(tokenize_yelp, batched=True)
tokenized_yelp.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Dataloaders
train_loader = DataLoader(tokenized_yelp["train"], batch_size=64, shuffle=True)
test_loader = DataLoader(tokenized_yelp["test"], batch_size=64)


d:\Thesis\iu_thesis_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
import torch
from tqdm import tqdm

def train_baseline_bert(model, dataloader, optimizer, device, num_epochs=3):
    loss_fn = CrossEntropyLoss()
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        print(f"\n🌗 Epoch {epoch + 1} — Standard CE Training")
        for batch in tqdm(dataloader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"✅ Epoch {epoch + 1} complete — Avg Loss: {avg_loss:.4f}")

    return model


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_baseline = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = AdamW(model_baseline.parameters(), lr=2e-5)

# Train
model_baseline = train_baseline_bert(model_baseline, train_loader, optimizer, device)

# Save model
model_baseline.save_pretrained("bert_baseline_yelp")
tokenizer.save_pretrained("bert_baseline_yelp")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🌗 Epoch 1 — Standard CE Training


Training: 100%|██████████| 782/782 [09:40<00:00,  1.35it/s]


✅ Epoch 1 complete — Avg Loss: 0.2016

🌗 Epoch 2 — Standard CE Training


Training: 100%|██████████| 782/782 [09:45<00:00,  1.34it/s]


✅ Epoch 2 complete — Avg Loss: 0.1117

🌗 Epoch 3 — Standard CE Training


Training: 100%|██████████| 782/782 [09:43<00:00,  1.34it/s]


✅ Epoch 3 complete — Avg Loss: 0.0635


('bert_baseline_yelp\\tokenizer_config.json',
 'bert_baseline_yelp\\special_tokens_map.json',
 'bert_baseline_yelp\\vocab.txt',
 'bert_baseline_yelp\\added_tokens.json')

In [12]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "D:/Thesis/bert_baseline_yelp"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

model.to(device)
model.eval() 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
evaluate_model(model,test_loader, device)
 


📊 Evaluation Results:
Accuracy:  0.9386
Precision: 0.9552
Recall:    0.9205
F1 Score:  0.9375
Confusion Matrix:
 [[18179   821]
 [ 1511 17489]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9233    0.9568    0.9397     19000
           1     0.9552    0.9205    0.9375     19000

    accuracy                         0.9386     38000
   macro avg     0.9392    0.9386    0.9386     38000
weighted avg     0.9392    0.9386    0.9386     38000



(0.9386315789473684,
 0.9374966496917716,
 0.9551611141452758,
 0.9204736842105263,
 array([[18179,   821],
        [ 1511, 17489]]))

Adversarial Attacks on Baseline Bert 

In [3]:
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import Dataset
from textattack import Attacker, AttackArgs
from transformers import BertTokenizer, BertForSequenceClassification

# Load your trained model
model_path = "D:/Thesis/bert_baseline_yelp"  
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Wrap it for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

TextFooler on Baseline BERT

In [2]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [2]:
# Step 1: Prepare dataset
from textattack.datasets import Dataset
from datasets import load_dataset

yelp_dataset = load_dataset("yelp_polarity")
sample_dataset = [(ex['text'], ex['label']) for ex in yelp_dataset['test'].select(range(100))]
ta_dataset = Dataset(sample_dataset)

# Step 2: Setup attack with POS constraint intact
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker, AttackArgs

attack = TextFoolerJin2019.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="textfooler_baseline_results.csv",
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path textfooler_baseline_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/100 [00:00<?, ?it/s]

From d:\Thesis\iu_thesis_env\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



From d:\Thesis\iu_thesis_env\Lib\site-packages\tensorflow_hub\resolver.py:120: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



...

--------------------------------------------- Result 97 ---------------------------------------------
[[0 (100%)]] --> [[1 (51%)]]

[[Clearly]] [[should]] have read a few [[reviews]] before going here. Whenever I go to these types of iconic regional dining establishments, I order the house specialty to best judge the place on its supposed merits. [[Since]] it was 11 in the morning, I chose not to order Primanti's #1 best seller, which the waitress told me was beer, but perhaps that would have made my experience better. As it was, I ordered the #2 best seller, the cheese steak. [[It]] was heinous. \n\nThink a [[grimy]], mechanically separated Salisbury-style steak with a [[meager]] [[slice]] of [[provolone]] on [[sliced]] [[Italian]] bread served signature Primanti-style: with [[coleslaw]], [[tomatoes]], and fries.\n\nFirst, the [[steak]] was disgusting: [[terrible]] flavor ([[smelled]] and tasted like hippie armpit, which I'm not [[necessarily]] [[opposed]] to as [[long]] as it's not [

[Succeeded / Failed / Skipped / Total] 95 / 0 / 3 / 98:  98%|█████████▊| 98/100 [17:29<00:21, 10.71s/it]

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (57%)]]

When there's this much hype about a place, it's hard to live up to, but they did a pretty damn [[good]] job. Maybe not the best sandwich I've ever had in my life, but if it was any random sandwich shop I'd happened to stumble into, it would definitely be five stars without a second thought.\n\nThe chili cheese fries were among the [[best]] I've ever had. I don't know what they do to them - it's not the chili that's particularly great, it's not the cheese that's particularly great, and it's not the fries that are particularly great. But somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very good despite the fact that I hate coleslaw. But they don't mean the typical mayo-and-carrot-and-purple-onion coleslaw; they make it with vinegar and [[oil]], so it's basically just like season

[Succeeded / Failed / Skipped / Total] 96 / 0 / 4 / 100: 100%|██████████| 100/100 [17:41<00:00, 10.62s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[1 (53%)]]

I've [[heard]] all the hype and [[wanted]] to [[try]] it.  [[Tonight]] was my [[second]] [[attempt]] here.   First time, new girl on the [[grill]] apologized to me at 10 minutes, at 15 I [[walked]].  Not even an [[acknowledgement]] from the wait [[staff]].   [[Tonight]], the waiter seemed mad that I didn't know what I wanted to order.  [[After]] I [[explained]] that I'd never been there, he brought me a menu and then [[proceeded]] to ignore me for 10 [[minutes]].  I [[walked]] again.  No more tries here.  Maybe I'm doing something wrong?  Haha.

I've [[listening]] all the hype and [[enjoyed]] to [[efforts]] it.  [[Ayer]] was my [[ii]] [[tempting]] here.   First time, new girl on the [[barbecue]] apologized to me at 10 minutes, at 15 I [[protested]].  Not even an [[homage]] from the wait [[job]].   [[Mornings]], the waiter seemed mad that I didn't know what

DeepWordBug (Character-Level Attack) on Baseline BERT

In [5]:
from textattack.attack_recipes import DeepWordBugGao2018
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset

# Prepare the dataset (if not already)
sample_dataset = [(ex['text'], ex['label']) for ex in yelp_dataset['test'].select(range(100))]
ta_dataset = Dataset(sample_dataset)

# Build the attack with the existing model wrapper
attack = DeepWordBugGao2018.build(model_wrapper)

# Attack arguments
attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="deepwordbug_baseline_results.csv",
    disable_stdout=False,
    random_seed=42
)

# Run the attack
attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path deepwordbug_baseline_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



--------------------------------------------- Result 1 ---------------------------------------------
[[1 (99%)]] --> [[0 (69%)]]

Contrary to other reviews, I have [[zero]] [[complaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. [[But]] here, my service and road coverage has all been well explained - and let up to me to [[decide]]. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.

Contrary to other reviews, I have [[zbro]] [[comphlaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to

--------------------------------------------- Result 2 ---------------------------------------------
[[0 (100%)]] --> [[1 (59%)]]

Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to [[fix]] a minor problem with a tire they put on. They \""fixed\"" it for free, and the very [[next]] [[morning]] I had the same issue. I called to complain, and the \""manager\"" didn't [[even]] apologize!!! So [[frustrated]]. [[Never]] going back.  [[They]] [[seem]] [[overpriced]], too.

Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to [[fxi]] a minor problem with a tire they put on. They \""fixed\"" it for free, and the very [[netx]] [[morniLng]] I had the same issue. I called to complain, and the \""manager\"" didn't [[egven]] apologize!!! So [[fruMtrated]]. [[NeveVr]] going back.  [[TZhey]] [[esem]] [[Averpriced]], too.




--------------------------------------------- Result 3 ---------------------------------------------
[[1 (96%)]] --> [[0 (99%)]]

[[Friendly]] staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long.

[[Fiendly]] staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long.




...

--------------------------------------------- Result 95 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

It wasn't bad, but it definitely wasn't good. \n\nAs soon as we walked in the door the place was jammed back with people who were not interested in seating anyone or telling anyone they would be seated. On one side of the restaurant there were literally two girls working, and one of them was pregnant. You can imagine the chaos. I wanted to have this \""Pittsburgh\"" experience that I've heard so much about but honestly I was more than underwhelmed. \n\nAfter being seated, it took the waitress 10 minutes to get around to actually saying hello to us and in between that time a large group of people decided to squeeze in a booth behind our table making my friend having to get up every time a waitress or another person needed to squeeze back there.\n\nThe sandwich at best was mediocre. The coleslaw was a bit too vinegary for my tastes and became sickening afte

--------------------------------------------- Result 97 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Clearly should have read a few reviews before going here. Whenever I go to these types of iconic regional dining establishments, I order the house specialty to best judge the place on its supposed merits. Since it was 11 in the morning, I chose not to order Primanti's #1 best seller, which the waitress told me was beer, but perhaps that would have made my experience better. As it was, I ordered the #2 best seller, the cheese steak. It was heinous. \n\nThink a grimy, mechanically separated Salisbury-style steak with a meager slice of provolone on sliced Italian bread served signature Primanti-style: with coleslaw, tomatoes, and fries.\n\nFirst, the steak was disgusting: terrible flavor (smelled and tasted like hippie armpit, which I'm not necessarily opposed to as long as it's not involved in something I'm trying to eat) with a gross, processed meat textur

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (98%)]]

[[When]] [[there's]] this much hype about a place, it's hard to live up to, but they did a pretty damn [[good]] job. [[Maybe]] not the best sandwich I've ever had in my life, but if it was any random sandwich shop I'd happened to stumble into, it would definitely be five stars without a second thought.\n\nThe chili cheese fries were among the [[best]] I've ever had. I don't know what they do to them - it's not the chili that's particularly great, it's not the cheese that's particularly great, and it's not the fries that are particularly great. [[But]] somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very good despite the fact that I hate coleslaw. But they don't mean the typical mayo-and-carrot-and-purple-onion coleslaw; they make it with vinegar and oil, so it's basically just




[Succeeded / Failed / Skipped / Total] 71 / 25 / 4 / 100: 100%|██████████| 100/100 [08:19<00:00,  5.00s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

I've heard all the hype and wanted to try it.  Tonight was my second attempt here.   First time, new girl on the grill apologized to me at 10 minutes, at 15 I walked.  Not even an acknowledgement from the wait staff.   Tonight, the waiter seemed mad that I didn't know what I wanted to order.  After I explained that I'd never been there, he brought me a menu and then proceeded to ignore me for 10 minutes.  I walked again.  No more tries here.  Maybe I'm doing something wrong?  Haha.


--------------------------------------------- Result 100 ---------------------------------------------
[[1 (99%)]] --> [[[SKIPPED]]]

This place holds a nostalgic appeal for people born and raised in Pittsburgh who grew up eating here. If that experience is what your looking for, please visit. If you're looking for a tasty meal, go somewhere else. 5 stars for history, 0 for f

Bert_Attack(MaskedLM) on Baseline Bert

In [3]:
def build_bert_attack_maskedlm(model_wrapper):
    from textattack.attack import Attack
    from textattack.constraints.semantics import WordEmbeddingDistance
    from textattack.constraints.pre_transformation import MaxModificationRate
    from textattack.goal_functions import UntargetedClassification
    from textattack.search_methods import GreedyWordSwapWIR
    from textattack.transformations import WordSwapMaskedLM

    # Transformation: masked language model with limited candidate pool
    transformation = WordSwapMaskedLM(
        method="bert-attack",
        max_candidates=15  # Reduce from 48 to prevent stalling
    )

    # Constraints: simplify and control modification budget
    constraints = [
        WordEmbeddingDistance(min_cos_sim=0.8),
        MaxModificationRate(max_rate=0.3)  # Limit to 30% of words modified
    ]

    # Objective
    goal_function = UntargetedClassification(model_wrapper)

    # Search method
    search_method = GreedyWordSwapWIR(wir_method="unk")

    return Attack(goal_function, constraints, transformation, search_method)

# Build it
attack = build_bert_attack_maskedlm(model_wrapper)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [6]:
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset

# Use a smaller input slice for quick runs
sample_dataset = [(ex['text'], ex['label']) for ex in yelp_dataset['test'].select(range(50))]
ta_dataset = Dataset(sample_dataset)

attack_args = AttackArgs(
    num_examples=50,
    log_to_csv="bertattack_baseline_maskedlm.csv",
    checkpoint_interval=5,
    checkpoint_dir="bertattack_baseline_ckpt",
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path bertattack_baseline_maskedlm.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  15
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): MaxModificationRate(
        (max_rate):  0.3
        (min_threshold):  1
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 1/50 [00:09<07:39,  9.37s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (99%)]] --> [[0 (61%)]]

Contrary [[to]] other [[reviews]], I have zero [[complaints]] about the service or the prices. [[I]] [[have]] been getting [[tire]] service here [[for]] the past 5 years [[now]], and compared to my experience with places like Pep Boys, these [[guys]] are experienced and know what they're doing. \nAlso, this is [[one]] place that [[I]] do not feel like I am being [[taken]] [[advantage]] of, [[just]] because of my gender. [[Other]] [[auto]] [[mechanics]] [[have]] been [[notorious]] for capitalizing on my ignorance [[of]] cars, [[and]] [[have]] sucked my bank account dry. [[But]] here, my service and road coverage has all been well explained - and [[let]] up [[to]] me [[to]] [[decide]]. \nAnd they just renovated the waiting room. It looks a lot better than it [[did]] in previous years.

Contrary [[of]] other [[review]], I have zero [[complaint]] about the servi

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   4%|▍         | 2/50 [00:23<09:17, 11.61s/it]

...

--------------------------------------------- Result 49 ---------------------------------------------
[[1 (100%)]] --> [[0 (51%)]]

Page's is a Pittsburgh [[icon]]. The ice cream is good, the selection is vast, and the price is right. They have so many different things to try, it's nearly impossible to choose. I'm a big fan of the cookie sundae, brownie sundae, and cinnamon roll sundae. The options at this place are pretty much endless. [[But]] sometimes a regular soft serve sugar cone is the way to go. It's better [[than]] Dairy Queen, AND you support a local business by going here. What more could you ask for. \n\nI don't know if I would say that Page's has particularly superior soft serve, but [[it's]] the experience and variety that makes this a four star place. I do think the ice cream is better than somewhere like Dairy Queen, but not by much. What makes Page's [[special]] is wide variety of options and soft serve flavors. It's also the nostalgia that goes along with it. [[It's]]

[Succeeded / Failed / Skipped / Total] 24 / 25 / 1 / 50: 100%|██████████| 50/50 [1:45:18<00:00, 126.37s/it]textattack: Saving checkpoint under "bertattack_baseline_ckpt\1748894271366.ta.chkpt" at 2025-06-03 01:27:51 after 50 attacks.


--------------------------------------------- Result 50 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

The first time I went here was with a group - we had reserved the entire restaurant for a retirement party.   I had a horrible experience.  Vowed never to go back!  At that time, I even had a terrible interaction with the owner.\n\nThis time I was invited with a couple of girlfriends because they love it. Decided to try again since it was more about the \""get together\"" than the food/location.\n\nThis time, overpriced food and bad service.  Overall, it was not anything out of the ordinary and not worth the price.   I had the veal speciality.   Large portion, but ok.  Staff was not very attentive for the small amount of customers present.\n\nI am a Pittsburgh native.  This is a bad neighborhood. \n\nI am NOT interested in ever going back!







[Succeeded / Failed / Skipped / Total] 24 / 25 / 1 / 50: 100%|██████████| 50/50 [1:45:18<00:00, 126.37s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 25     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 98.0%  |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 48.98% |
| Average perturbed word %:     | 13.57% |
| Average num. words per input: | 143.44 |
| Avg num queries:              | 289.29 |
+-------------------------------+--------+


BERT_ATTACK(Embedding) on Baseline BERT

In [1]:
from textattack.attack import Attack
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.pre_transformation import MaxModificationRate
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding

def build_bert_attack_embedding(model_wrapper):
    transformation = WordSwapEmbedding(max_candidates=15)
    constraints = [
        WordEmbeddingDistance(min_cos_sim=0.8),
        MaxModificationRate(max_rate=0.3)
    ]
    goal_function = UntargetedClassification(model_wrapper)
    search_method = GreedyWordSwapWIR(wir_method="unk")
    return Attack(goal_function, constraints, transformation, search_method)


d:\Thesis\iu_thesis_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from textattack.datasets import Dataset
from textattack.models.wrappers import HuggingFaceModelWrapper

# Load 100 samples
sample_yelp = [(ex["text"], ex["label"]) for ex in yelp_dataset["test"].select(range(100))]
ta_dataset = Dataset(sample_yelp)

# Wrap your trained model
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

from textattack import AttackArgs, Attacker

attack = build_bert_attack_embedding(model_wrapper)

attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="bertattack_baseline_embedding_results.csv",
    checkpoint_dir="embedding_baseline_ckpt",
    checkpoint_interval=10,
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()



textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path bertattack_baseline_embedding_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): MaxModificationRate(
        (max_rate):  0.3
        (min_threshold):  1
      )
  (is_black_box):  True
) 



--------------------------------------------- Result 1 ---------------------------------------------
[[1 (99%)]] --> [[0 (60%)]]

Contrary to other reviews, I have [[zero]] [[complaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - [[and]] let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.

Contrary to other reviews, I have [[zilch]] [[allegations]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my

--------------------------------------------- Result 2 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had the same issue. I called to complain, and the \""manager\"" didn't even apologize!!! So frustrated. Never going back.  They seem overpriced, too.




--------------------------------------------- Result 3 ---------------------------------------------
[[1 (96%)]] --> [[0 (98%)]]

[[Friendly]] staff, same starbucks fair [[you]] get anywhere else.  Sometimes the lines can get long.

[[Amity]] staff, same starbucks fair [[vous]] get anywhere else.  Sometimes the lines can get long.




...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (61%)]]

[[When]] there's this much hype about a place, it's hard to live up to, [[but]] they did a pretty damn [[good]] job. [[Maybe]] not the best sandwich I've ever had in my life, but if it was any random sandwich shop I'd happened to stumble into, it would definitely be five stars without a second thought.\n\nThe chili cheese fries were among the [[best]] I've ever had. I don't know what they do to them - it's not the chili that's particularly great, it's not the cheese that's particularly great, and it's not the fries that are particularly great. [[But]] somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very good despite the fact that I hate coleslaw. But they don't mean the typical mayo-and-carrot-and-purple-onion coleslaw; they make it with vinegar and oil, [[so]] it's basically 




textattack: Saving checkpoint under "embedding_baseline_ckpt\1748947924302.ta.chkpt" at 2025-06-03 16:22:04 after 100 attacks.


--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

I've heard all the hype and wanted to try it.  Tonight was my second attempt here.   First time, new girl on the grill apologized to me at 10 minutes, at 15 I walked.  Not even an acknowledgement from the wait staff.   Tonight, the waiter seemed mad that I didn't know what I wanted to order.  After I explained that I'd never been there, he brought me a menu and then proceeded to ignore me for 10 minutes.  I walked again.  No more tries here.  Maybe I'm doing something wrong?  Haha.


--------------------------------------------- Result 100 ---------------------------------------------
[[1 (99%)]] --> [[[SKIPPED]]]

This place holds a nostalgic appeal for people born and raised in Pittsburgh who grew up eating here. If that experience is what your looking for, please visit. If you're looking for a tasty meal, go somewhere else. 5 stars for history, 0 for f

[Succeeded / Failed / Skipped / Total] 79 / 17 / 4 / 100: 100%|██████████| 100/100 [10:55<00:00,  6.56s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 79     |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 96.0%  |
| Accuracy under attack:        | 17.0%  |
| Attack success rate:          | 82.29% |
| Average perturbed word %:     | 10.62% |
| Average num. words per input: | 140.67 |
| Avg num queries:              | 263.75 |
+-------------------------------+--------+


TextBugger on Baseline BERT

In [7]:
from textattack.attack_recipes import TextBuggerLi2018
from textattack import AttackArgs, Attacker
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import Dataset

# Wrap LNPL model
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Build attack
attack = TextBuggerLi2018.build(model_wrapper)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [8]:
# Select test set
sample_yelp = [(ex["text"], ex["label"]) for ex in yelp_dataset["test"].select(range(100))]
ta_dataset = Dataset(sample_yelp)

attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="textbugger_baseline_results.csv",
    checkpoint_dir="textbugger_baseline_ckpt",
    checkpoint_interval=10,
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()


textattack: Logging to CSV at path textbugger_baseline_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  0%|          | 0/100 [00:00<?, ?it/s]

From d:\Thesis\iu_thesis_env\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



From d:\Thesis\iu_thesis_env\Lib\site-packages\tensorflow_hub\resolver.py:120: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (57%)]]

When [[there's]] this much hype about a place, it's hard to live up to, but they did a pretty [[damn]] [[good]] [[job]]. Maybe not the best [[sandwich]] I've ever had in my life, but if it was any random sandwich [[shop]] I'd happened to stumble into, it would definitely be five stars without a second thought.\n\nThe chili cheese fries were among the best [[I've]] ever had. I don't know what they do to them - it's not the [[chili]] that's particularly great, it's not the [[cheese]] that's particularly great, and it's not the fries that are particularly great. But somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very good despite the fact that I hate [[coleslaw]]. But they don't mean the typical [[mayo-and-carrot-and-purple-onion]] coleslaw; they make it with vinegar and [[oil]]

[Succeeded / Failed / Skipped / Total] 79 / 17 / 4 / 100: 100%|██████████| 100/100 [11:01<00:00,  6.61s/it]textattack: Saving checkpoint under "textbugger_baseline_ckpt\1748950366216.ta.chkpt" at 2025-06-03 17:02:46 after 100 attacks.


--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

I've heard all the hype and wanted to try it.  Tonight was my second attempt here.   First time, new girl on the grill apologized to me at 10 minutes, at 15 I walked.  Not even an acknowledgement from the wait staff.   Tonight, the waiter seemed mad that I didn't know what I wanted to order.  After I explained that I'd never been there, he brought me a menu and then proceeded to ignore me for 10 minutes.  I walked again.  No more tries here.  Maybe I'm doing something wrong?  Haha.


--------------------------------------------- Result 100 ---------------------------------------------
[[1 (99%)]] --> [[[SKIPPED]]]

This place holds a nostalgic appeal for people born and raised in Pittsburgh who grew up eating here. If that experience is what your looking for, please visit. If you're looking for a tasty meal, go somewhere else. 5 stars for history, 0 for f

[Succeeded / Failed / Skipped / Total] 79 / 17 / 4 / 100: 100%|██████████| 100/100 [11:01<00:00,  6.61s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 79     |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 96.0%  |
| Accuracy under attack:        | 17.0%  |
| Attack success rate:          | 82.29% |
| Average perturbed word %:     | 27.06% |
| Average num. words per input: | 140.67 |
| Avg num queries:              | 294.84 |
+-------------------------------+--------+


Training LNPL on noisy Yelp Data

In [4]:
import torch
from torch.optim import AdamW  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_lnpl(
    model,
    dataloader,
    optimizer,
    device,
    num_epochs=3,
    lambda_nt=0.04,
    margin=0.5,
    class_weights=None
):
    import torch
    from torch.nn import CrossEntropyLoss
    import torch.nn.functional as F
    from tqdm import tqdm
    import random

    loss_fn = CrossEntropyLoss(weight=class_weights) if class_weights is not None else CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        total_loss_pt = 0.0
        total_loss_nt = 0.0
        total_seen = 0

        print(f"\n🌗 LNPL Epoch {epoch + 1}: Joint PT + NT Phase")

        for batch in tqdm(dataloader, desc="Training (LNPL full)"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = F.softmax(logits, dim=1)
            batch_size = labels.size(0)

            # === Positive Training Loss (PT) ===
            pt_loss = loss_fn(logits, labels)

            # === Negative Training Loss (NT) ===
            comp_labels = torch.empty_like(labels)
            for i in range(batch_size):
                choices = [lbl for lbl in range(logits.size(1)) if lbl != labels[i].item()]
                comp_labels[i] = random.choice(choices)

            pos_scores = logits.gather(1, labels.unsqueeze(1)).squeeze()
            neg_scores = logits.gather(1, comp_labels.unsqueeze(1)).squeeze()
            nt_losses = F.relu(margin + neg_scores - pos_scores)
            nt_loss = nt_losses.mean()

            # === Combined Loss ===
            total_loss = pt_loss + lambda_nt * nt_loss

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            total_loss_pt += pt_loss.item()
            total_loss_nt += nt_loss.item()
            total_seen += batch_size

        avg_pt_loss = total_loss_pt / (len(dataloader) + 1e-8)
        avg_nt_loss = total_loss_nt / (len(dataloader) + 1e-8)

        print(f"✅ Epoch {epoch + 1} complete — Avg PT Loss: {avg_pt_loss:.4f} | Avg NT Loss: {avg_nt_loss:.4f}")

    print("\n✅ LNPL (paper-faithful) training complete.")
    return model


In [5]:
from datasets import load_dataset
from collections import Counter

# Load Yelp Polarity dataset
yelp_dataset = load_dataset("yelp_polarity")

# Count class labels in training set
label_counts = Counter(yelp_dataset["train"]["label"])
print(label_counts)


Counter({0: 280000, 1: 280000})


In [7]:
from datasets import load_dataset, DatasetDict, concatenate_datasets
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW  
from torch.nn import CrossEntropyLoss
import torch

# Load Yelp Polarity dataset and create a balanced 50k subset (25k per class)
yelp_dataset = load_dataset("yelp_polarity")

class_0 = yelp_dataset["train"].filter(lambda ex: ex["label"] == 0).select(range(25000))
class_1 = yelp_dataset["train"].filter(lambda ex: ex["label"] == 1).select(range(25000))
balanced_train = concatenate_datasets([class_0, class_1])
balanced_yelp = DatasetDict({"train": balanced_train, "test": yelp_dataset["test"]})

# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_yelp(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_yelp = balanced_yelp.map(tokenize_yelp, batched=True)
tokenized_yelp.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# DataLoaders
yelp_train_loader = DataLoader(tokenized_yelp["train"], batch_size=64, shuffle=True)
yelp_test_loader = DataLoader(tokenized_yelp["test"], batch_size=64)

In [ ]:
# Model setup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and class weights
optimizer = AdamW(model.parameters(), lr=2e-5)
class_weights = torch.tensor([0.5, 0.5]).to(device)

# ✅ Final LNPL training
train_lnpl(
    model=model,
    dataloader=yelp_train_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=3,  # Adjust if needed
    lambda_nt=0.04,
    margin=0.5,
    class_weights=class_weights
)

# ==== Save Model After Training ====
model_save_path = "bert_lnpl_yelp"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"\n✅ Model and tokenizer saved to '{model_save_path}'")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🌗 LNPL Epoch 1: Joint PT + NT Phase


Training (LNPL full): 100%|██████████| 782/782 [09:25<00:00,  1.38it/s]


✅ Epoch 1 complete — Avg PT Loss: 0.2037 | Avg NT Loss: 0.1523

🌗 LNPL Epoch 2: Joint PT + NT Phase


Training (LNPL full): 100%|██████████| 782/782 [09:28<00:00,  1.38it/s]


✅ Epoch 2 complete — Avg PT Loss: 0.1127 | Avg NT Loss: 0.0866

🌗 LNPL Epoch 3: Joint PT + NT Phase


Training (LNPL full): 100%|██████████| 782/782 [09:30<00:00,  1.37it/s]


✅ Epoch 3 complete — Avg PT Loss: 0.0593 | Avg NT Loss: 0.0467

✅ LNPL (paper-faithful) training complete.

✅ Model and tokenizer saved to 'bert_lnpl_yelp_final'


In [8]:
from transformers import BertForSequenceClassification, BertTokenizer

model_path = "D:/Thesis/bert_lnpl_yelp"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

model.to(device)
model.eval() 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [9]:
evaluate_model(model, yelp_test_loader, device)



📊 Evaluation Results:
Accuracy:  0.9392
Precision: 0.9391
Recall:    0.9393
F1 Score:  0.9392
Confusion Matrix:
 [[17842  1158]
 [ 1153 17847]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9393    0.9391    0.9392     19000
           1     0.9391    0.9393    0.9392     19000

    accuracy                         0.9392     38000
   macro avg     0.9392    0.9392    0.9392     38000
weighted avg     0.9392    0.9392    0.9392     38000



(0.9391842105263158,
 0.9391922115511117,
 0.9390686661404893,
 0.9393157894736842,
 array([[17842,  1158],
        [ 1153, 17847]]))

In [ ]:
# Load your trained LNPL model
model_path = "D:/Thesis/bert_lnpl_yelp"  # or wherever you saved it
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Wrap it for TextAttack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)


[nltk_data] Downloading package punkt to C:\Users\solom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\solom/nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\solom/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to C:\Users\solom/nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\solom/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\solom/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\solom/nltk_data...

In [10]:
import nltk
import os

# Explicitly set path
custom_nltk_path = "D:/Thesis/iu_thesis_env/Lib/site-packages/nltk_data"
os.environ["nltk_data"] = custom_nltk_path
nltk.data.path.clear()
nltk.data.path.append(custom_nltk_path)

# Force reload tagger
from nltk import word_tokenize, pos_tag

sentence = "The food was surprisingly good but the service was slow."
tokens = word_tokenize(sentence)
tags = pos_tag(tokens, tagset='universal')
print(tags)


[('The', 'DET'), ('food', 'NOUN'), ('was', 'VERB'), ('surprisingly', 'ADV'), ('good', 'ADJ'), ('but', 'CONJ'), ('the', 'DET'), ('service', 'NOUN'), ('was', 'VERB'), ('slow', 'ADJ'), ('.', '.')]


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetDict, concatenate_datasets

# Load and balance Yelp
yelp_dataset = load_dataset("yelp_polarity")
class_0 = yelp_dataset["train"].filter(lambda ex: ex["label"] == 0).select(range(25000))
class_1 = yelp_dataset["train"].filter(lambda ex: ex["label"] == 1).select(range(25000))
balanced_train = concatenate_datasets([class_0, class_1])
balanced_yelp = DatasetDict({"train": balanced_train, "test": yelp_dataset["test"]})

# Tokenize
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_yelp(example): return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
tokenized_yelp = balanced_yelp.map(tokenize_yelp, batched=True)
tokenized_yelp.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Dataloaders
train_loader = DataLoader(tokenized_yelp["train"], batch_size=64, shuffle=True)
test_loader = DataLoader(tokenized_yelp["test"], batch_size=64)

Text Fooler

In [15]:
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker, AttackArgs


In [4]:
# Step 1: Prepare dataset
from textattack.datasets import Dataset
from datasets import load_dataset

yelp_dataset = load_dataset("yelp_polarity")
sample_dataset = [(ex['text'], ex['label']) for ex in yelp_dataset['test'].select(range(100))]
ta_dataset = Dataset(sample_dataset)

# Step 2: Setup attack with POS constraint intact
from textattack.attack_recipes import TextFoolerJin2019
from textattack import Attacker, AttackArgs

attack = TextFoolerJin2019.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="textfooler_lnpl_results.csv",
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path textfooler_lnpl_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/100 [00:00<?, ?it/s]

From d:\Thesis\iu_thesis_env\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



From d:\Thesis\iu_thesis_env\Lib\site-packages\tensorflow_hub\resolver.py:120: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



...

--------------------------------------------- Result 97 ---------------------------------------------
[[0 (100%)]] --> [[1 (53%)]]

[[Clearly]] [[should]] have [[read]] a few reviews before going here. Whenever I go to these types of iconic regional dining establishments, I [[order]] the house [[specialty]] to best [[judge]] the place on its supposed merits. Since it was 11 in the [[morning]], I chose not to order Primanti's #1 best [[seller]], which the waitress told me was beer, but perhaps that would have made my [[experience]] better. As it was, I ordered the #2 best seller, the cheese steak. It was [[heinous]]. \n\nThink a [[grimy]], mechanically separated Salisbury-style steak with a meager slice of [[provolone]] on sliced Italian bread [[served]] signature Primanti-style: with [[coleslaw]], tomatoes, and fries.\n\nFirst, the steak was disgusting: terrible flavor (smelled and tasted like hippie armpit, which I'm not necessarily opposed to as long as it's not involved in something

[Succeeded / Failed / Skipped / Total] 93 / 2 / 3 / 98:  98%|█████████▊| 98/100 [16:32<00:20, 10.13s/it]

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (50%)]]

When there's this much hype about a place, it's hard to live up to, but they did a pretty damn [[good]] [[job]]. Maybe not the [[best]] sandwich I've ever had in my life, but if it was any random sandwich shop I'd happened to stumble into, it would definitely be five [[stars]] without a second thought.\n\nThe chili cheese fries were among the best I've ever had. I don't know what they do to them - it's not the [[chili]] that's particularly great, it's not the cheese that's particularly great, and it's not the fries that are particularly great. But somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very good [[despite]] the fact that I hate coleslaw. But they don't mean the typical mayo-and-carrot-and-purple-onion coleslaw; they make it with vinegar and oil, so it's basically just

[Succeeded / Failed / Skipped / Total] 94 / 2 / 4 / 100: 100%|██████████| 100/100 [16:43<00:00, 10.03s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[1 (55%)]]

I've heard all the hype and wanted to try it.  Tonight was my [[second]] [[attempt]] here.   [[First]] [[time]], [[new]] girl on the grill apologized to me at 10 [[minutes]], at 15 I [[walked]].  Not even an [[acknowledgement]] from the wait [[staff]].   [[Tonight]], the waiter seemed mad that I didn't know what [[I]] [[wanted]] to order.  After [[I]] explained that I'd never been there, he brought me a menu and then proceeded to [[ignore]] me for 10 [[minutes]].  I walked again.  [[No]] more tries here.  [[Maybe]] I'm doing something wrong?  Haha.

I've heard all the hype and wanted to try it.  Tonight was my [[ii]] [[wish]] here.   [[Begin]] [[tiempo]], [[novo]] girl on the grill apologized to me at 10 [[record]], at 15 I [[was]].  Not even an [[acknowledgment]] from the wait [[gov]].   [[Ayer]], the waiter seemed mad that I didn't know what [[me]] [[wan

DeepWordBug (Character-Level Attack)

In [5]:
from textattack.attack_recipes import DeepWordBugGao2018
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset

# Prepare the dataset (if not already)
sample_dataset = [(ex['text'], ex['label']) for ex in yelp_dataset['test'].select(range(100))]
ta_dataset = Dataset(sample_dataset)

# Build the attack with the existing model wrapper
attack = DeepWordBugGao2018.build(model_wrapper)

# Attack arguments
attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="deepwordbug_lnpl_results.csv",
    disable_stdout=False,
    random_seed=42
)

# Run the attack
attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path deepwordbug_lnpl_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:01<02:57,  1.79s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[0 (57%)]]

Contrary to other reviews, I have zero [[complaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and [[know]] what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been [[notorious]] for capitalizing on my ignorance of cars, and have sucked my bank account dry. [[But]] here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.

Contrary to other reviews, I have zero [[comphlaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   2%|▏         | 2/100 [00:05<04:31,  2.77s/it]

...

--------------------------------------------- Result 97 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Clearly should have read a few reviews before going here. Whenever I go to these types of iconic regional dining establishments, I order the house specialty to best judge the place on its supposed merits. Since it was 11 in the morning, I chose not to order Primanti's #1 best seller, which the waitress told me was beer, but perhaps that would have made my experience better. As it was, I ordered the #2 best seller, the cheese steak. It was heinous. \n\nThink a grimy, mechanically separated Salisbury-style steak with a meager slice of provolone on sliced Italian bread served signature Primanti-style: with coleslaw, tomatoes, and fries.\n\nFirst, the steak was disgusting: terrible flavor (smelled and tasted like hippie armpit, which I'm not necessarily opposed to as long as it's not involved in something I'm trying to eat) with a gross, processed meat textur

[Succeeded / Failed / Skipped / Total] 67 / 28 / 3 / 98:  98%|█████████▊| 98/100 [08:05<00:09,  4.96s/it]

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (71%)]]

[[When]] [[there's]] this much hype about a place, it's hard to live up to, but they did a pretty damn [[good]] [[job]]. Maybe not the best sandwich I've ever had in my life, but if it was any random sandwich shop I'd happened to stumble into, it would definitely be five [[stars]] without a second thought.\n\nThe chili cheese fries were among the [[best]] I've ever had. I don't know what they do to them - it's not the chili that's particularly great, it's not the cheese that's particularly great, and it's not the fries that are particularly great. [[But]] somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very good despite the fact that I hate coleslaw. But they don't mean the typical mayo-and-carrot-and-purple-onion coleslaw; they make it with vinegar and oil, so it's basically 

[Succeeded / Failed / Skipped / Total] 68 / 28 / 4 / 100: 100%|██████████| 100/100 [08:09<00:00,  4.90s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[1 (51%)]]

[[I've]] heard all the hype and wanted to try it.  Tonight was my [[second]] [[attempt]] here.   [[First]] time, [[new]] girl on the grill [[apologized]] to me at [[10]] [[minutes]], at [[15]] I [[walked]].  [[Not]] [[even]] an acknowledgement from the [[wait]] [[staff]].   [[Tonight]], the waiter seemed mad that I didn't [[know]] what I wanted to [[order]].  After I explained that I'd never been there, he brought me a menu and then [[proceeded]] to [[ignore]] me for [[10]] minutes.  I walked again.  [[No]] more tries here.  [[Maybe]] I'm doing something wrong?  Haha.

[[Iv'e]] heard all the hype and wanted to try it.  Tonight was my [[secsnd]] [[attempe]] here.   [[FirsSt]] time, [[niew]] girl on the grill [[apolgized]] to me at [[K10]] [[Jinutes]], at [[51]] I [[waTlked]].  [[oNt]] [[evne]] an acknowledgement from the [[awit]] [[stIaff]].   [[Townight]],

BERT-Attacks on the LNPL Model

In [ ]:
from textattack.attack import Attack
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapMaskedLM
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import MaxModificationRate


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


BERT-Attack_MaskedLM

In [ ]:
def build_bert_attack_maskedlm(model_wrapper):
    from textattack.attack import Attack
    from textattack.constraints.semantics import WordEmbeddingDistance
    from textattack.constraints.pre_transformation import MaxModificationRate
    from textattack.goal_functions import UntargetedClassification
    from textattack.search_methods import GreedyWordSwapWIR
    from textattack.transformations import WordSwapMaskedLM

    # Transformation: masked language model with limited candidate pool
    transformation = WordSwapMaskedLM(
        method="bert-attack",
        max_candidates=15  # Reduce from 48 to prevent stalling
    )

    # Constraints: simplify and control modification budget
    constraints = [
        WordEmbeddingDistance(min_cos_sim=0.8),
        MaxModificationRate(max_rate=0.3)  # Limit to 30% of words modified
    ]

    # Objective
    goal_function = UntargetedClassification(model_wrapper)

    # Search method
    search_method = GreedyWordSwapWIR(wir_method="unk")

    return Attack(goal_function, constraints, transformation, search_method)

# Build it
attack = build_bert_attack_maskedlm(model_wrapper)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [15]:
from textattack import Attacker, AttackArgs
from textattack.datasets import Dataset

# Use a smaller input slice for quick runs
sample_dataset = [(ex['text'], ex['label']) for ex in yelp_dataset['test'].select(range(50))]
ta_dataset = Dataset(sample_dataset)

attack_args = AttackArgs(
    num_examples=50,
    log_to_csv="bertattack_lnpl_no_use.csv",
    checkpoint_interval=5,
    checkpoint_dir="bertattack_ckpt",
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()

textattack: Logging to CSV at path bertattack_lnpl_no_use.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bert-attack
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  15
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): MaxModificationRate(
        (max_rate):  0.3
        (min_threshold):  1
      )
  (is_black_box):  True
) 



--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[0 (55%)]]

Contrary [[to]] other reviews, [[I]] have zero [[complaints]] about the service or the prices. [[I]] have been getting tire service here for the past 5 years [[now]], and compared to my experience with places like Pep Boys, these [[guys]] are experienced and know what they're [[doing]]. \[[nAlso]], this is one place that I do not feel like I am being taken advantage of, [[just]] because of my gender. Other [[auto]] [[mechanics]] have been [[notorious]] for capitalizing on my ignorance [[of]] cars, and have sucked my bank account dry. [[But]] here, my service and road coverage has all been well explained - and let up to me to [[decide]]. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.

Contrary [[of]] other reviews, [[1]] have zero [[complaint]] about the service or the prices. [[1]] have been getting tire ser

--------------------------------------------- Result 2 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Last summer I had an appointment to get new tires and had to wait a super long time. I also went in this week for them to fix a minor problem with a tire they put on. They \""fixed\"" it for free, and the very next morning I had the same issue. I called to complain, and the \""manager\"" didn't even apologize!!! So frustrated. Never going back.  They seem overpriced, too.




--------------------------------------------- Result 3 ---------------------------------------------
[[1 (98%)]] --> [[[FAILED]]]

Friendly staff, same starbucks fair you get anywhere else.  Sometimes the lines can get long.




...

--------------------------------------------- Result 49 ---------------------------------------------
[[1 (100%)]] --> [[0 (50%)]]

[[Page's]] is a Pittsburgh [[icon]]. The ice cream is [[good]], the [[selection]] is [[vast]], [[and]] the [[price]] is right. They [[have]] [[so]] [[many]] different things to [[try]], [[it's]] [[nearly]] impossible to choose. I'm a big fan of the cookie [[sundae]], [[brownie]] sundae, and cinnamon roll [[sundae]]. The options at this place are pretty much endless. But sometimes a [[regular]] soft [[serve]] sugar cone is the [[way]] to go. It's better [[than]] Dairy Queen, AND you support a local business by going here. What more [[could]] you [[ask]] for. \n\nI [[don't]] know if I would say that [[Page's]] has particularly superior soft serve, [[but]] it's the experience and variety that makes this a four [[star]] place. I do think the ice cream is better than somewhere like Dairy Queen, but not by much. What makes Page's [[special]] is [[wide]] variety 







textattack: Saving checkpoint under "bertattack_ckpt\1747824860720.ta.chkpt" at 2025-05-21 16:24:20 after 50 attacks.


--------------------------------------------- Result 50 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

The first time I went here was with a group - we had reserved the entire restaurant for a retirement party.   I had a horrible experience.  Vowed never to go back!  At that time, I even had a terrible interaction with the owner.\n\nThis time I was invited with a couple of girlfriends because they love it. Decided to try again since it was more about the \""get together\"" than the food/location.\n\nThis time, overpriced food and bad service.  Overall, it was not anything out of the ordinary and not worth the price.   I had the veal speciality.   Large portion, but ok.  Staff was not very attentive for the small amount of customers present.\n\nI am a Pittsburgh native.  This is a bad neighborhood. \n\nI am NOT interested in ever going back!







[Succeeded / Failed / Skipped / Total] 24 / 25 / 1 / 50: 100%|██████████| 50/50 [1:30:12<00:00, 108.25s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 25     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 98.0%  |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 48.98% |
| Average perturbed word %:     | 14.51% |
| Average num. words per input: | 143.44 |
| Avg num queries:              | 292.73 |
+-------------------------------+--------+


BERT-Attack Embedding

In [16]:
from textattack.attack import Attack
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.pre_transformation import MaxModificationRate
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding

def build_bert_attack_embedding(model_wrapper):
    transformation = WordSwapEmbedding(max_candidates=15)
    constraints = [
        WordEmbeddingDistance(min_cos_sim=0.8),
        MaxModificationRate(max_rate=0.3)
    ]
    goal_function = UntargetedClassification(model_wrapper)
    search_method = GreedyWordSwapWIR(wir_method="unk")
    return Attack(goal_function, constraints, transformation, search_method)


In [22]:
from textattack.datasets import Dataset
from textattack.models.wrappers import HuggingFaceModelWrapper

# Load 100 samples
sample_yelp = [(ex["text"], ex["label"]) for ex in yelp_dataset["test"].select(range(100))]
ta_dataset = Dataset(sample_yelp)

# Wrap your trained model
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)


In [2]:
from textattack import AttackArgs, Attacker

attack = build_bert_attack_embedding(model_wrapper)

attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="bertattack_embedding_results.csv",
    checkpoint_dir="embedding_ckpt",
    checkpoint_interval=10,
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()


AttributeError: partially initialized module 'torch' has no attribute 'version' (most likely due to a circular import)

TextBugger

In [19]:
from textattack.attack_recipes import TextBuggerLi2018
from textattack import AttackArgs, Attacker
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import Dataset

# Wrap LNPL model
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Build attack
attack = TextBuggerLi2018.build(model_wrapper)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
# Select test set
sample_yelp = [(ex["text"], ex["label"]) for ex in yelp_dataset["test"].select(range(100))]
ta_dataset = Dataset(sample_yelp)

attack_args = AttackArgs(
    num_examples=100,
    log_to_csv="textbugger_lnpl_results.csv",
    checkpoint_dir="textbugger_ckpt",
    checkpoint_interval=10,
    disable_stdout=False,
    random_seed=42
)

attacker = Attacker(attack, ta_dataset, attack_args)
attacker.attack_dataset()


textattack: Logging to CSV at path textbugger_lnpl_results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:05<09:36,  5.83s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (100%)]] --> [[0 (51%)]]

Contrary to other reviews, I have [[zero]] [[complaints]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and [[know]] what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. [[But]] here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.

Contrary to other reviews, I have [[zilch]] [[complaintѕ]] about the service or the prices. I have been getting tire service here for the past 5 years now, and compared t

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   2%|▏         | 2/100 [00:10<08:42,  5.34s/it]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[1 (100%)]] --> [[0 (53%)]]

When there's this much hype about a place, it's hard to live up to, but they did a pretty damn [[good]] [[job]]. Maybe not the [[best]] sandwich I've ever had in my life, but if it was any random sandwich shop I'd happened to stumble into, it would definitely be five [[stars]] without a second thought.\n\nThe chili cheese fries were among the best I've ever had. I don't know what they do to them - it's not the [[chili]] that's particularly great, it's not the cheese that's particularly great, and it's not the fries that are particularly great. But somehow, putting them all together just turns it into unbeatable [[awesome]].\n\nThe sandwich itself (I got the corned beef) was very [[good]] [[despite]] the fact that I hate coleslaw. But they don't mean the typical mayo-and-carrot-and-purple-onion coleslaw; they make it with vinegar and oil, so it's basically 

[Succeeded / Failed / Skipped / Total] 80 / 16 / 4 / 100: 100%|██████████| 100/100 [11:37<00:00,  6.98s/it]textattack: Saving checkpoint under "textbugger_ckpt\1747831391509.ta.chkpt" at 2025-05-21 18:13:11 after 100 attacks.


--------------------------------------------- Result 99 ---------------------------------------------
[[0 (100%)]] --> [[1 (54%)]]

I've heard all the hype and wanted to try it.  Tonight was my [[second]] [[attempt]] here.   [[First]] [[time]], new girl on the grill apologized to me at 10 minutes, at 15 [[I]] walked.  [[Not]] [[even]] an [[acknowledgement]] from the wait [[staff]].   [[Tonight]], the waiter seemed mad that I didn't [[know]] what [[I]] wanted to order.  After I [[explained]] that I'd never been there, he [[brought]] me a menu and then proceeded to [[ignore]] me for 10 minutes.  [[I]] walked again.  No more [[tries]] here.  [[Maybe]] I'm doing something wrong?  Haha.

I've heard all the hype and wanted to try it.  Tonight was my [[segundo]] [[attem]] [[pt]] here.   [[Fіrst]] [[ti]] [[me]], new girl on the grill apologized to me at 10 minutes, at 15 [[li]] walked.  [[N]] [[ot]] [[eevn]] an [[recognition]] from the wait [[staf𝚏]].   [[Evening]], the waiter seemed mad that 

[Succeeded / Failed / Skipped / Total] 80 / 16 / 4 / 100: 100%|██████████| 100/100 [11:37<00:00,  6.98s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 80     |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 96.0%  |
| Accuracy under attack:        | 16.0%  |
| Attack success rate:          | 83.33% |
| Average perturbed word %:     | 28.74% |
| Average num. words per input: | 140.67 |
| Avg num queries:              | 302.89 |
+-------------------------------+--------+


Shap Visual Generation 

In [22]:
import shap
import torch
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification

# Load model and tokenizer
model_path = "D:/Thesis/bert_lnpl_yelp"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")
device = next(model.parameters()).device

# Your prediction function (already fixed)
def predict_prob(texts):
    if not isinstance(texts, list):
        texts = list(texts)
    texts = [str(t) for t in texts]

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy()

    return probs

# Set up SHAP explainer
explainer = shap.Explainer(predict_prob, tokenizer)

# Save folder
output_dir = "D:/Thesis/shap_output"
os.makedirs(output_dir, exist_ok=True)

# List of examples (original and adversarial)
examples = [
    {
        "attack": "TextFooler",
        "original": "This Valentines Day I ordered a pizza for my boyfriend and asked that they make a heart on it out of green peppers. The pizza was great, the heart was perfect, and he loved it!",
        "perturbed": "These Valentin Day me directs a pizza for my bridegroom and asks that they make a heart on it out of ecological diced. The pizzeria was beautiful, the heart was perfect, and he loved it!"
    },{
        "attack": "TextFooler",
        "original": "Love this beer distributor.  They always have what I'm looking for. The workers are extremely nice and always willing to help. Best one I've seen by far.",
        "perturbed": "Honey this brews distributor.  They always ai what I'm searches for. The working are extremely super and always ready to enabled. Best one I've noted by far."
    },{
        "attack": "DeepWordBug",
        "original": "got the buffet pizza was good they also give you free soup to .",
        "perturbed": "got the buffet pizza was ogod they also give you frle soup to ."
    },{
        "attack": "DeepWordBug",
        "original": "Love this beer distributor.  They always have what I'm looking for. The workers are extremely nice and always willing to help. Best one I've seen by far.",
        "perturbed": "LEove this bere disrtibutor.  They atlways have what Im' looking for. Tyhe workes are extremel nce and always willMing to hlep. Best noe I've Xseen by far."
    },{
        "attack": "BERT-Attack (Masked LM)",
        "original": "This is my go-to distributer. They have a great selection and lots of hard to find brews. I haven't noticed that it's any more expensive than going elsewhere but honestly if I don't buy here then I'm buying six packs which as we all know are a huuugggge rip off. So any time I have a case of something I love I feel like I'm getting a bargain. The cold selection does suck so I try to plan ahead and have what I need already cold. Staff here are all very helpful- and will have recommendations if you ask.",
        "perturbed": "This is my go-far distributer. They have a large selections and lots of hard to find brews. I haven't noticed that it't any more costly than going elsewhere but truly if i don'non buy here then i're purchasing six packs which as we all know are a huuugggge rip off. So every times I had a case of something I loved I feel like i't getting a bargain. The cold selection does suck therefore I try to scheme ahead and had what I need already cold. Staff here are all very helpful, and will have recommendations if you request."
    },{
        "attack": "BERT-Attack (Masked LM)",
        "original": "When scoping around online for a good NYE dinner destination for my husband and me, Casbah's prix fixe menu really jumped off the page! Since the Big Burrito group is so popular, I was happy to get reservations, especially since a lot of times were already booked.\n\nThe hubby and I arrived a few minutes early for dinner. I had planned additional time to park and was delightfully reminded that Big Burrito restaurants have free valet parking -- so great, especially with slushy snow on the ground! Even though we were a few minutes early, the hostess took our coats and seated us immediately.  \n\nService was extremely attentive from start to finish. Waters were re-filled quickly. Bread was brought to the table almost immediately and we were offered another basket as soon as we finished the first. The bread came with a delicious spread - was it goat cheese? - that I just couldn't stop eating. Silverware whisked away with each course was promptly replaced.\n\nWe ordered both the land and sea courses to share so we could try the entire NYE prix fixe menu. None of the eight dishes disappointed! The first courses were Sea Scallops & Octopus and Duck Confit. The sea scallop was perfectly cooked, the octopus - thought not my cup of tea! - was also well cooked, and the balsamic flavor in the dish was very tasty. I absolutely loved the duck confit dish. It was served with mustard greens and fruit with a to-die-for dressing - a delicious combination!\n\nThe second courses were Potato Gnocchi and Mushroom Tortelloni. The gnocchi was served with lobster, which was a unique but nice paring. Though a strongly flavored dish, the tortelloni was delicious, and I enjoyed another unique paring, tortelloni with chunks of beef short rib.\n\nLake Ontario Walleye and Roasted Veal Strip Loin were the third courses. Both were cooked to perfection. The walleye's skin was perfectly crispy, and the veal strip loin was cooked medium well but was still deliciously juicy and tender. However, a component of each dish was disappointing -- a mustard flavor in the walleye and pickled mustard seeds covering the veal. I really can't hold this against Casbah though. I'm just not a fan of mustard!\n\nFinally, the desserts was fantastic! The Chocolate Ganache Tart was the ending of the sea courses. The chocolate was strong, even a little bitter, and was served with a little champagne strawberry compote. I grew more accustomed to the bitterness with each bite, and it was the perfect ending to three courses filled with fish. A Caramel Macaroon completed the land courses. The macaroon was delicious and served with a little apple sorbet and caramel sauce.\n\nOur experience at Casbah was spectacular! Based on this experience, I debated giving the restaurant five stars... but decided to wait until I returned a second time. And I can't wait to return!",
        "perturbed": "When scoping around online for a good NYE dinner destination for my husband and me, Casbah's prix fixe menu really jumped off the page! Since the Big Burrito group is so popular, I was happy to get reservations, especially since a lot of times were already booked.\n\nThe hubby and I arrived a few minutes early for dinner. I had planned additional time to park and was delightfully reminded that Big Burrito restaurants have free valet parking -- so great, especially with slushy snow on the ground! Even though we were a few minutes early, the hostess took our coats and seated us immediately.  \n\nService was extremely attenresponsible from start to finish. Waters were re-filled quickly. Bread was brought to the table almost immediately and we were offered another basket as soon as we finished the first. The bread came with a delicious spread - was it goat cheese? - that I just couldn't stop eating. Silverware whisked away with each course was promptly replaced.\n\nWe ordered both the land and sea courses to share so we could try the entire NYE prix fixe menu. no to the eight dishes disappointed! The first courses were Sea Scallops & Octopus and Duck Confit. The sea scallop was perfectly cooked, the octopus - thought not my cup of tea! - was also well cooked, and the balsamic flavor in the dish was very tasty. I absolutely loved the duck confit dish. It was served with mustard greens and fruit with a to-die-for dressing - a delicious combination!\n\nThe second courses were Potato Gnocchi and Mushroom Tortelloni. The gnocchi was served with lobster, which was a unique but nice paring. Though a strongly flavored dish, the tortelloni was delicious, and I enjoyed another unique paring, tortelloni with chunks of beef short rib.\n\nLake Ontario Walleye and Roasted Veal Strip Loin were the third courses. Both were cooked to perfection. The walleye's skin was perfectly crispy, and the veal strip loin was cooked medium well but was still deliciously juicy and tender. However, a component of each dish was disappointing -- a mustard flavor in the walleye and pickled mustard seeds covering the veal. I really can't hold this against Casbah though. I'm just not a fan of mustard!\n\nFinally, the desserts was fantastic! The Chocolate Ganache Tart was the ending of the sea courses. The chocolate was strong, even a little bitter, and was served with a little champagne strawberry compote. I grew more accustomed to the bitterness with each bite, and it was the perfect ending to three courses filled with fish. A Caramel Macaroon completed the land courses. The macaroon was delicious and served with a little apple sorbet and caramel sauce.\n\nOur experience at Casbah was spectacular! Based on this experience, I debated giving the restaurant five stars... but decided to wait until I returned a second time. And I can't wait to return!"
    },{
        "attack": "BERT-Attack (Embedding)",
        "original": "My girlfriend and I went to Casbah for the first time recently and we both enjoyed our experience.\n\nThe atmosphere is dressy, but the restaurant isn't so formal that it feels stuffy or uncomfortable, which is a welcome change compared to most classy restaurants.  The environment was relaxed, and it was easy to have a quiet conversation throughout the meal.  As for the meal itself, we had a reservation and were promptly seated when we showed up.\n\nOur waiter was courteous and provided good, fast service without being overbearing or constantly hovering, which was quite nice.  For an appetizer, we ordered the cheese tray which was tasty as well as fun and unique.\n\nFollowing the cheese tray, I dined on the cioppino while my girlfriend had the casereccia.  Both dishes were tasty and our seafood was skillfully cooked, particularly the sea bass in my dish which was delicious.\n\nI'd recommend this restaurant for anyone looking to celebrate an occasion or have some good, out of the ordinary food.",
        "perturbed": "My girlfriend and I went to Casbah for the first time recently und we both liked our experience.\n\nThe atmosphere is dressy, but the restaurant isn't so official that it feels stuffy orr uncomfortable, which is a salute change compared to most classy restaurants.  The environment was loosened, and it was easy to have a quiet conversation throughout the meal.  As for the meal himself, we has a booking and were promptly seated when we showed up.\n\nOur waiter was courteous and provided good, fast service sans being overbearing or constantly hovering, which was quite pleasant.  For an appetizer, we ordered the cheese tray which was tasty as well as fun and unique.\n\nFollowing the cheese tray, I dined on the cioppino while my girlfriend had the casereccia.  Both dishes were tasty and our seafood was skillfully cooked, particularly the sea bass in my dish which was appetizing.\n\nI'd recommends this restaurant for somebody search to celebrate an occasion or have some good, out of the ordinary nutrition."
    },{
        "attack": "BERT-Attack (Embedding)",
        "original": "Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.",
        "perturbed": "Contrary to other reviews, I have nil grievance about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced und savoir what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been proverbial for capitalizing on my ignorance of cars, and have sucked my bank account dry. Although here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years."
    },{
        "attack": "TextBugger",
        "original": "Every time I have been to Casbah for lunch, brunch, or dinner, the food has been great and the service has been impeccable.  Sure, I've had more exceptionally delicious and unique meals in Pittsburgh, but Casbah has Consistent Awesomeness down pat.  Just get a reservation ahead of time.  Also, the desserts are mandatory!  \n\nThe duck confit gnocchi is smashing-- it's a nice balance of slightly sweet and very savory--and my favorite is probably the Casbah Double-Cut Pork Chop: super-yum comfort food.  The short rib ravioli is some pot-roasty Goodness, and the long island duck is another great choice.  \n\nA nice surprise here are the great happy hour specials at the bar.  Try the Boyd & Blair cocktail and the sangria.",
        "perturbed":"Eveⲅy deadline I have been to Casablanca for dinners, dinner, or dinոer, the food has been great and the serves has been faultless.  Convinced, I've had more exceptionally delicious and uniԛue dining in Pittsbuⲅgh, but Casablanca has Consonant Awesomeness down so.  Just ge𝚝 a reservation ahead of deadline.  Also, the desserts are obligatory!  \n\nThe duck confit gnocchi is smashinɡ-- it's a nice balance of slightly sweet and very savory--aոd my preferred is presumably the Casbah Double-Cut Swine Cut: supеr-yum comfort food.  To shot rb ravioli is some po-roasty Gooness, and the long island duck is else great choices.  \n\nA nice surprise here are the great happy hour specials at the br.  Try the Byd & Blair cocktail and the sangⲅia."
    },{
        "attack": "TextBugger",
        "original": "Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years.",
        "perturbed": "Contrary to other reviews, I have zilch complaintѕ about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and knowing what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. Although here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years."
    }

]
# Run and plot
for i, ex in enumerate(examples, start=1):
    for label, text in [("original", ex["original"]), ("perturbed", ex["perturbed"])]:
        attack = ex["attack"].replace(" ", "_").replace("(", "").replace(")", "")
        filename = f"shap_{attack}_example{i}_{label}.html"
        filepath = os.path.join(output_dir, filename)

        # Get SHAP value and render as HTML string
        shap_values = explainer([text])
        pred_class = predict_prob([text]).argmax()

        # Use the SHAP explanation for that class only
        shap_value = shap_values[:, :, pred_class][0]

        html= shap.plots.text(shap_value, display=False)

        # Save HTML to disk
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(html)


        print(f"✅ Saved: {filepath}")


✅ Saved: D:/Thesis/shap_output\shap_TextFooler_example1_original.html
✅ Saved: D:/Thesis/shap_output\shap_TextFooler_example1_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_TextFooler_example2_original.html
✅ Saved: D:/Thesis/shap_output\shap_TextFooler_example2_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_DeepWordBug_example3_original.html
✅ Saved: D:/Thesis/shap_output\shap_DeepWordBug_example3_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_DeepWordBug_example4_original.html
✅ Saved: D:/Thesis/shap_output\shap_DeepWordBug_example4_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Masked_LM_example5_original.html


Token indices sequence length is longer than the specified maximum sequence length for this model (630 > 512). Running this sequence through the model will result in indexing errors


✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Masked_LM_example5_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Masked_LM_example6_original.html


✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Masked_LM_example6_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Embedding_example7_original.html
✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Embedding_example7_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Embedding_example8_original.html
✅ Saved: D:/Thesis/shap_output\shap_BERT-Attack_Embedding_example8_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_TextBugger_example9_original.html
✅ Saved: D:/Thesis/shap_output\shap_TextBugger_example9_perturbed.html
✅ Saved: D:/Thesis/shap_output\shap_TextBugger_example10_original.html
✅ Saved: D:/Thesis/shap_output\shap_TextBugger_example10_perturbed.html
